# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets

import json
import datetime as dt
import random
from scipy import stats
from citipy import citipy
from scipy.stats import linregress
from pprint import pprint


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
Cities_List = pd.read_csv(f"weather_df.csv")

Cities_List.head(1)


,Latitude,Longitude,City,Country Code,Temp,Temp Max,Temp Min,City ID,Wind Speed,Humidity,Cloudiness,Date
0,31.6315,-8.0083,marrakesh,ma,102.27,102.27,102.27,2542997.0,6.91,21.0,40.0,07/21/2021


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [45]:

#this sets the height and width
figure_layout = {"width": "400px",
                "height": "250px",
                "border": "3px solid black",
                "padding":"3px"}

#this removes the rows or records with missing data
cities_list_clean = Cities_List.dropna()
len(cities_list_clean)

#this creates a figure using gmaps, and configuring using the aforementioned height and width
#fig = gmaps.figure(layout=figure_layout)
center_coordinates = (40.75, 1.00)


fig = gmaps.figure(center=center_coordinates, zoom_level=2, map_type="HYBRID")
#gmaps.figure(layout=figure_layout)

#this takes the latitude and longitude values for each city listed on cities_list_clean
lat_lng = cities_list_clean[["Latitude", "Longitude"]]

#this sets each humdity value as a "float", which allows it to use the decimal point
humidity =  cities_list_clean["Humidity"].astype(float)

#this explains that the ceiling for the heat map is the "max_humidity" value
max_humidity = humidity.max()


heatLayer_humidity = gmaps.heatmap_layer(lat_lng, weights = humidity, 
                                         dissipating=True, max_intensity=max_humidity, point_radius = 10)

fig.add_layer(heatLayer_humidity)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [57]:
cities_list_clean = cities_list_clean.dropna()
cities_list_clean

fit_weather = cities_list_clean.loc[(cities_list_clean["Temp Max"]>65)&
                                   (cities_list_clean["Temp Min"]<85)&
                                   (cities_list_clean["Wind Speed"]<20)&
                                   (cities_list_clean["Humidity"]<80)&
                                   (cities_list_clean["Cloudiness"]<25)]

fit_weather

,Latitude,Longitude,City,Country Code,Temp,Temp Max,Temp Min,City ID,Wind Speed,Humidity,Cloudiness,Date
3,49.5000,-115.0687,fernie,ca,73.83,75.29,73.60,5952370.0,6.53,30.0,15.0,07/21/2021
6,69.4541,-133.0374,tuktoyaktuk,ca,77.00,77.00,77.00,6170031.0,8.05,50.0,20.0,07/21/2021
29,62.9088,6.9145,bud,no,59.86,65.07,53.91,7626370.0,16.28,77.0,17.0,07/21/2021
35,-29.8333,-51.1500,sapucaia,br,64.71,74.41,62.19,3448031.0,1.99,50.0,0.0,07/21/2021


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [61]:
#this converts the name to "hotel_df"
hotel_df = fit_weather

#this makes it a dataframe
hotel_df = pd.DataFrame(hotel_df)

#this adds an empty column called "Hotel Name"
hotel_df["Hotel Name"] = ""

hotel_df

,Latitude,Longitude,City,Country Code,Temp,Temp Max,Temp Min,City ID,Wind Speed,Humidity,Cloudiness,Date,Hotel Name
3,49.5000,-115.0687,fernie,ca,73.83,75.29,73.60,5952370.0,6.53,30.0,15.0,07/21/2021,
6,69.4541,-133.0374,tuktoyaktuk,ca,77.00,77.00,77.00,6170031.0,8.05,50.0,20.0,07/21/2021,
29,62.9088,6.9145,bud,no,59.86,65.07,53.91,7626370.0,16.28,77.0,17.0,07/21/2021,
35,-29.8333,-51.1500,sapucaia,br,64.71,74.41,62.19,3448031.0,1.99,50.0,0.0,07/21/2021,


In [120]:
HotelName = []
HotelAddress = []
HotelLatitude = []
HotelLongitude = []
HotelRating = []

#this goes through each row and saves the value in each row's specified column into the list above
for index, record in hotel_df.iterrows():
    ThisCity = record["City"]
    ThisCountry = record["Country Code"]
    ThisCoordinate = f"{record['Latitude']}, {record['Longitude']}"
    Radius = 5000
    Type = "hotel"
    Keyword = "hotel"
    
    #this establishes the parameters for the google search
    parameters = {"location":ThisCoordinate,
                 "radius":Radius,
                 "type":Type,
                 "keyword":Keyword,
                 "key":g_key}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=parameters).json()
    results = response['results']

    try:
        HotelName.append(results[0]["name"])
        HotelAddress.append(results[0]["vicinity"])
        HotelLatitude.append(results[0]["geometry"]["location"]["lat"])
        HotelLongitude.append(results[0]["geometry"]["location"]["lng"])
        HotelRating.append(results[0]["rating"])
        print(f"Found {Type}(s) in {ThisCity}")
    
    except:
        HotelName.append(np.nan)
        HotelAddress.append(np.nan)
        HotelLatitude.append(np.nan)
        HotelLongitude.append(np.nan)
        HotelRating.append(np.nan)
        Print(f"No {Type}s found in {ThisCity}")
        
results

Found hotel(s) in fernie
Found hotel(s) in tuktoyaktuk
Found hotel(s) in bud
Found hotel(s) in sapucaia


[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': -29.8412056, 'lng': -51.176033},
   'viewport': {'northeast': {'lat': -29.83977837010728,
     'lng': -51.17464647010728},
    'southwest': {'lat': -29.84247802989272, 'lng': -51.17734612989273}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
  'name': 'Hotel CHA Klein Ville INN - Rio Grande do Sul - Brasil',
  'photos': [{'height': 1536,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/101924532772238043848">Klein Ville Hotel</a>'],
    'photo_reference': 'Aap_uEAVzqVh9DL0SSiogx4wdkWQqBuJfMzIYvIXHui2tJisZuHbBWMaJdby6MJ_NHdlkWYJRcRrIIhsREfUe6mTYqVS4Gq4P6TEY0TN63RuNUqXdd0fbVk1C7w_i1eKU-5X6puWFBU45JYZ3sEWy8CSb52pFAJ5FQOPFBnxvhO213fcLb_N',
    'width': 2048}],
  'place_id': 'ChIJx3_2L1lvGZURdnzuNyA2OeU',
  'plus_code': {'compound_code': '5R5F+GH Vila Osorio, Esteio - State of Rio Grande do Sul, Brazil',
   'global_code': '582C5R5F+GH'},
  'rating': 3.9,

In [121]:
hotel_df["Hotel Name"] = HotelName
hotel_df["Hotel Address"] = HotelAddress
hotel_df["Hotel Latitude"] = HotelLatitude
hotel_df["Hotel Longitude"] = HotelLongitude

hotel_results_df = hotel_df.loc[:,["Hotel Latitude", "Hotel Longitude", 
                                   "Hotel Address", "Hotel Name", "City"]]

hotel_results_df = pd.DataFrame(hotel_results_df)

hotel_results_df = hotel_results_df.dropna()

hotel_results_df

,Hotel Latitude,Hotel Longitude,Hotel Address,Hotel Name,City
3,49.514389,-115.057721,"1622 7 Ave, Fernie",Best Western Plus Fernie Mountain Lodge,fernie
6,69.442135,-133.023790,"608 Kitti Road, Tuktoyaktuk",Smitty's Bed and Breakfast,tuktoyaktuk
29,62.907040,6.902697,"Vikaveien 165, Bud",Lisjesundet,bud
35,-29.841206,-51.176033,"R. Tiradentes, 286 - Vila Osorio, Esteio",Hotel CHA Klein Ville INN - Rio Grande do Sul ...,sapucaia


In [ ]:
fig = gmaps.figure(center=center_coordinates, zoom_level=2)

#this takes the latitude and longitude values for each city listed on cities_list_clean
hotel_lat_lng = hotel_results_df[["Hotel Latitude", "Hotel Longitude"]]

#this sets each humdity value as a "float", which allows it to use the decimal point
hotel_coordinates =  hotel_lat_lng["Humidity"].astype(float)

#this explains that the ceiling for the heat map is the "max_humidity" value
max_humidity = humidity.max()


heatLayer_humidity = gmaps.heatmap_layer(lat_lng, weights = humidity, 
                                         dissipating=True, max_intensity=max_humidity, point_radius = 10)

fig.add_layer(heatLayer_humidity)

fig

In [62]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Country'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
